In [23]:
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
    

In [7]:
display(HTML("<style>.container { width:100% !important; }</style>"))

In [34]:
df = pd.read_csv('data/plays.csv')
df = df.set_index('Dataline')
df[df['Play'] == 'King Lear']

df['ActSceneLine'] = df[['ActSceneLine']].applymap(lambda x: str(x))

df['Act'] = [ np.nan if line == np.nan else line.split('.')[0] for line in df.ActSceneLine]
df['Scene'] = [ np.nan if line == np.nan else line.split('.')[1] if len(line.split('.')) > 1 else np.nan for line in df.ActSceneLine]
df

,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine,Act,Scene
Dataline,,,,,,,
1,Henry IV,NaN,nan,NaN,ACT I,nan,NaN
2,Henry IV,NaN,nan,NaN,SCENE I. London. The palace.,nan,NaN
3,Henry IV,NaN,nan,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ...",nan,NaN
4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,",1,1
5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,",1,1
6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils,1,1
7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.,1,1
8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil,1,1
9,Henry IV,1.0,1.1.6,KING HENRY IV,Shall daub her lips with her own children's bl...,1,1


In [17]:
df.Play.unique()

array(['Henry IV', 'Henry VI Part 1', 'Henry VI Part 2',
       'Henry VI Part 3', 'Alls well that ends well', 'As you like it',
       'Antony and Cleopatra', 'A Comedy of Errors', 'Coriolanus',
       'Cymbeline', 'Hamlet', 'Henry V', 'Henry VIII', 'King John',
       'Julius Caesar', 'King Lear', 'Loves Labours Lost', 'macbeth',
       'Measure for measure', 'Merchant of Venice',
       'Merry Wives of Windsor', 'A Midsummer nights dream',
       'Much Ado about nothing', 'Othello', 'Pericles', 'Richard II',
       'Richard III', 'Romeo and Juliet', 'Taming of the Shrew',
       'The Tempest', 'Timon of Athens', 'Titus Andronicus',
       'Troilus and Cressida', 'Twelfth Night', 'Two Gentlemen of Verona',
       'A Winters Tale'], dtype=object)